<a href="https://colab.research.google.com/github/osman-mo94/Sarcopenia-NLP-project/blob/main/synthetic_letters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and import packages

In [104]:
!pip install docx2txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [105]:
!pip install spacy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [106]:
!pip install negspacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [107]:
!pip install scispacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [108]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz (120.2 MB)


In [109]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import docx2txt
import spacy
from spacy.matcher import PhraseMatcher
from spacy.pipeline import EntityRuler
from negspacy.negation import Negex
from negspacy.termsets import termset
from spacy.tokens import Span
import scispacy
from scispacy.abbreviation import AbbreviationDetector
from spacy import displacy

In [110]:
#Initialize nlp pipeline with scispacy model (for processing biomedical, scientific and clinical text)
nlp = spacy.load("en_ner_bc5cdr_md")
#Add abbreviation detector for medical abbreviations
nlp.add_pipe("abbreviation_detector")

In [111]:
#View components of nlp pipeline
nlp.component_names

['tok2vec',
 'tagger',
 'attribute_ruler',
 'lemmatizer',
 'parser',
 'ner',
 'abbreviation_detector']

In [112]:
#Mount google drive to access required files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Import letters for analysis

In [113]:
#Import letters (note that these letters do not refer to real patients)
letter_A = docx2txt.process('/content/drive/MyDrive/NLP projects/Dummy letters/dummy letters/Letter A.docx')
letter_B = docx2txt.process('/content/drive/MyDrive/NLP projects/Dummy letters/dummy letters/Letter B.docx')

In [114]:
print(letter_A)

Mr A Smith

567 Ghengis Khan Drive, Newcastle NE4 5XX



Diagnoses:

Poor mobility due to chronic pain, low confidence and previous falls 

Weight loss, anaemia and raised inflammatory markers of unknown aetiology 

Low mood secondary to poor mobility 

Breathlessness and elevated BNP awaiting echo 

Chronic back pain with degenerative changes on MRI 

Urinary frequency and incontinence 



Other diagnoses:

Complex partial epileptic seizures

Hypertension 

Osteoarthritis with bilateral total hip replacements

Atrial fibrillation 

Asthma 

Patent foramen ovale 

Previous cerebellar stroke 



Medications:

Atorvastatin

Docusate

Ferrous fumarate

Vitamin-D3

Furosemide

Gaviscon

Flutiform inhaler

Salbutamol inhaler

Lansoprazole

Losartan 

Paracetamol 

Phenytoin

Tegretol slow release

Codeine

Warfarin



Suggested changes to medication 

Reduce codeine 15 mg dose but try and use regularly 2-3 times per day 



Follow up arrangements

 I will organise ultrasound of the abdomen 

In [115]:
print(letter_B)

Mrs B Smith

Flat 1, Farringdon Road, Newcastle NE2 5DH

Date of Birth: 01/01/1932





Diagnoses: 

Falls due to gait and balance disorder 

Hyperthyroidism due to thyroxine over-replacement

Sarcopenia 

Orthostatic hypotension 



Existing diagnoses: 

Hypertension 

Hypothyroidism 

Vitamin B12 deficiency 

Previous fractured wrist

Visual impairment due to cataracts 



Medications: 

Alendronic acid 

Calcium and vitamin-D 

Bendroflumethiazide 

Vitamin B12 

Simvastatin 

Ramipril 



Medication changes: 

Please reduce thyroxine dose to 75mcg once daily



Follow up arrangements: 

I will write back when I see the results of her 24 hour electrocardiogram



For Primary care 

Could you please forward me a copy of the 24 hour blood pressure monitor that she says she had in your surgery recently? 



I saw Mrs Smith for a face-to-face appointment at the Belsay Clinic today; she was accompanied by her son. She gives a history of two falls over the last few months and feels unstea

In [116]:
#Apply nlp pipeline to letter A
doc_A = nlp(letter_A)

/usr/local/lib/python3.7/dist-packages/scispacy/abbreviation.py:230: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


In [117]:
#Apply nlp pipeline to letter B
doc_B = nlp(letter_B)

/usr/local/lib/python3.7/dist-packages/scispacy/abbreviation.py:230: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


# Apply sci-spacy NER

In [118]:
#Visualise entities in Letter A
displacy.render(doc_A, style='ent', jupyter = True)

# Build PhraseMatcher

In [119]:
#Define a list of terms indicative of muscle weakness
weakness_list = ["muscle weakness", "weak", "uses a stick", "uses a walking stick", 
                    "uses a frame", "uses a zimmer frame", "uses a walker", "uses a walking aid",
                    "furniture walks", "difficulty mobilising", "difficulty walking", "wheelchair"
                    "difficulty standing", "difficulty climbing stairs", "cannot climb stairs", "housebound",
                    "bedbound", "hoist transfer", "slowed up", "limited mobility", "poor mobility"
                    "needs assistance", "difficulty carrying", "falls", "fallen",
                    "found on floor", "long lie"]

In [120]:
#Initialize matcher
matcher = PhraseMatcher(nlp.vocab)

#Apply spaCy nlp pipeline to list of weakness terms
weakness_terms = [nlp(i) for i in weakness_list]


/usr/local/lib/python3.7/dist-packages/scispacy/abbreviation.py:230: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


In [121]:
#Add weakness terms to PhraseMatcher
matcher.add("WEAKNESS TERM", weakness_terms)

In [122]:
#Add pattern for SARC-F score
sarcf_list = ["SARC-F", "SARC F", "SARCF", "sarc-f", "sarc f", "Sarc f", "Sarc F", "Sarc-f", "Sarc-F"]

sarcf_terms = [nlp(i) for i in sarcf_list]

matcher.add("SARC-F", sarcf_terms)


/usr/local/lib/python3.7/dist-packages/scispacy/abbreviation.py:230: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


In [123]:
#Add pattern for Sarcopenia diagnosis
sarcopenia_diagnosis = ["Sarcopenia", "sarcopenia"]

sarcopenia_terms = [nlp(i) for i in sarcopenia_diagnosis]

#Add to matcher
matcher.add("Sarcopenia", sarcopenia_terms)


/usr/local/lib/python3.7/dist-packages/scispacy/abbreviation.py:230: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


In [124]:
#Apply matcher to letter A
matchesA = matcher(doc_A)

for match_id, start, end in matchesA: 
  span = doc_A[start:end]
  match_id_string = nlp.vocab.strings[match_id]
  print("Match:",match_id_string, "-", span.text, "( Location = ", start, end, ")")

Match: WEAKNESS TERM - falls ( Location =  27 28 )
Match: WEAKNESS TERM - falls ( Location =  249 250 )
Match: WEAKNESS TERM - fallen ( Location =  297 298 )
Match: WEAKNESS TERM - housebound ( Location =  315 316 )
Match: SARC-F - SARC-F ( Location =  976 977 )


In [125]:
#Apply matcher to letter B
matchesB = matcher(doc_B)

for match_id, start, end in matchesB: 
  span = doc_B[start:end]
  match_id_string = nlp.vocab.strings[match_id]
  print("Match:",match_id_string, "-", span.text, "( Location = ", start, end, ")")

Match: Sarcopenia - Sarcopenia ( Location =  37 38 )
Match: WEAKNESS TERM - falls ( Location =  172 173 )
Match: WEAKNESS TERM - falls ( Location =  203 204 )
Match: SARC-F - Sarc-F ( Location =  532 533 )
Match: WEAKNESS TERM - falls ( Location =  664 665 )
Match: Sarcopenia - sarcopenia ( Location =  675 676 )
Match: WEAKNESS TERM - falls ( Location =  720 721 )
Match: Sarcopenia - sarcopenia ( Location =  790 791 )


Limited matches from using the PhraseMatcher, rule-based matcher would likely be more sensitive. An expanded list of terms indicating weakness would also be helpful.

# Try a rule-based matcher

In [126]:
#Import rule-based matcher
from spacy.matcher import Matcher

In [127]:
#Initialize matcher
rb_matcher = Matcher(nlp.vocab)

#Add patterns for weakness
weakness_pattern = [
                    [{"LEMMA": "fall"}], [{"LEMMA": "weak"}], [{"LOWER": "housebound"}], [{"LOWER": "bedbound"}],
                    [{"LEMMA": "use"}, {"LOWER": "a", "OP": "?"}, {"LEMMA": "walk", "OP": "?"}, {"LOWER": "stick"}],
                    [{"LEMMA": "use"}, {"LOWER": "a", "OP": "?"}, {"LEMMA": "walk", "OP": "?"}, {"LOWER": "zimmer", "OP": "?"}, {"LOWER": "frame"}],
                    [{"LEMMA": "use"}, {"LOWER": "a", "OP": "?"}, {"LEMMA": "walk"}, {"LOWER": "aid", "OP": "?"}],
                    [{"LOWER": "furniture"}, {"LEMMA": "walk"}], [{"LEMMA": "difficult"}, {"LEMMA": "walk"}],
                    [{"LEMMA": "difficult"}, {"LEMMA": "mobilise"}], [{"LEMMA": "difficult"}, {"LEMMA": "stand"}],
                    [{"LEMMA": "difficult"}, {"LOWER": "with", "OP": "?"}, {"LEMMA": "climb", "OP": "?"}, {"LEMMA": "stair"}],
                    [{"LOWER": "cannot"}, {"LEMMA": "climb", "OP": "?"}, {"LEMMA": "stair"}],
                    [{"LOWER": "can't"}, {"LEMMA": "climb", "OP": "?"}, {"LEMMA": "stair"}],
                    [{"LEMMA": "hoist"}, {"LEMMA": "transfer"}], [{"LEMMA": "slow"}, {"LOWER": "up"}],
                    [{"LEMMA": "limit"}, {"LOWER": "mobility"}],  [{"LOWER": "poor"}, {"LOWER": "mobility"}],
                    [{"LEMMA": "need"}, {"LEMMA": "assist"}], [{"LEMMA": "require"}, {"LEMMA": "assist"}],
                    [{"LEMMA": "difficult"}, {"LEMMA": "carry"}], [{"LOWER": "found"}, {"LOWER": "on"}, {"LOWER": "floor"}],
                    [{"LOWER": "long"}, {"LOWER": "lie"}], [{"LEMMA": "lack"}, {"LOWER": "of", "OP": "?"}, {"LOWER": "mobility"}],
                    [{"LEMMA": "lack"}, {"LOWER": "of", "OP": "?"}, {"LEMMA": "strength"}]
]


#Add patternS to matcher
rb_matcher.add("WEAKNESS TERM", weakness_pattern)


In [128]:
#Add pattern for extracting SARC-F score: 

sarcf_pattern = [
                 [{"LOWER": "sarc-f"}, {"LOWER": "score", "OP": "?"}, {"LOWER": "of", "OP": "?"}, {"LIKE_NUM": True}],
                 [{"LOWER": "sarc-f"}, {"LOWER": "score", "OP": "?"}, {"LOWER": "is", "OP": "?"}, {"LIKE_NUM": True}],
                 [{"LOWER": "sarc-f"}, {"IS_PUNCT": True, "OP": "?"}, {"LIKE_NUM": True}],
                 [{"LOWER": "sarcf"}, {"LOWER": "score", "OP": "?"}, {"LOWER": "of", "OP": "?"}, {"LIKE_NUM": True}],
                 [{"LOWER": "sarcf"}, {"LOWER": "score", "OP": "?"}, {"LOWER": "is", "OP": "?"}, {"LIKE_NUM": True}],
                 [{"LOWER": "sarcf"}, {"IS_PUNCT": True, "OP": "?"}, {"LIKE_NUM": True}],
                 [{"LOWER": "sarc"}, {"LOWER": "score", "OP": "?"}, {"LOWER": "of", "OP": "?"}, {"LIKE_NUM": True}],
                 [{"LOWER": "sarc"}, {"LOWER": "score", "OP": "?"}, {"LOWER": "is", "OP": "?"}, {"LIKE_NUM": True}],
                 [{"LOWER": "sarc"}, {"IS_PUNCT": True, "OP": "?"}, {"LIKE_NUM": True}],
                 [{"LOWER": "sarc f"}, {"LOWER": "score", "OP": "?"}, {"LOWER": "of", "OP": "?"}, {"LIKE_NUM": True}],
                 [{"LOWER": "sarc f"}, {"LOWER": "score", "OP": "?"}, {"LOWER": "is", "OP": "?"}, {"LIKE_NUM": True}],
                 [{"LOWER": "sarc f"}, {"IS_PUNCT": True, "OP": "?"}, {"LIKE_NUM": True}],
]

#Add SARC-F pattern to matcher
rb_matcher.add("SARC-F", sarcf_pattern)

In [129]:
#Apply rb_matcher to letter A

rb_matchesA = rb_matcher(doc_A)

for match_id, start, end in rb_matchesA: 
  span = doc_A[start:end]
  match_id_string = nlp.vocab.strings[match_id]
  print("Match:",match_id_string, "-", span.text, "( Location = ", start, end, ")")

Match: WEAKNESS TERM - Poor mobility ( Location =  16 18 )
Match: WEAKNESS TERM - falls ( Location =  27 28 )
Match: WEAKNESS TERM - poor mobility ( Location =  45 47 )
Match: WEAKNESS TERM - poor mobility ( Location =  246 248 )
Match: WEAKNESS TERM - falls ( Location =  249 250 )
Match: WEAKNESS TERM - fallen ( Location =  297 298 )
Match: WEAKNESS TERM - lack of mobility ( Location =  305 308 )
Match: WEAKNESS TERM - housebound ( Location =  315 316 )
Match: WEAKNESS TERM - lacks strength ( Location =  357 359 )
Match: WEAKNESS TERM - falling ( Location =  364 365 )
Match: WEAKNESS TERM - fall ( Location =  392 393 )
Match: SARC-F - SARC-F score of 10/10 ( Location =  976 980 )
Match: WEAKNESS TERM - lack of strength ( Location =  1092 1095 )


In [130]:
#Apply rb_matcher to letter B
rb_matchesB = rb_matcher(doc_B)

for match_id, start, end in rb_matchesB: 
  span = doc_B[start:end]
  match_id_string = nlp.vocab.strings[match_id]
  print("Match:",match_id_string, "-", span.text, "( Location = ", start, end, ")")

Match: WEAKNESS TERM - Falls ( Location =  23 24 )
Match: WEAKNESS TERM - falls ( Location =  172 173 )
Match: WEAKNESS TERM - falls ( Location =  203 204 )
Match: WEAKNESS TERM - fell ( Location =  225 226 )
Match: WEAKNESS TERM - fell ( Location =  325 326 )
Match: SARC-F - Sarc-F score of 7/10 ( Location =  532 536 )
Match: WEAKNESS TERM - falling ( Location =  584 585 )
Match: WEAKNESS TERM - fell ( Location =  603 604 )
Match: WEAKNESS TERM - falls ( Location =  664 665 )
Match: WEAKNESS TERM - falls ( Location =  720 721 )


The rule-based matcher works better at detecting phrases rather than using a simple PhraseMatcher:

*   12 matches for letter A, vs. 4 matches using Phrasematcher
*   9 matches for letter B, vs. 4 matches using Phrasematcher




# Entity-ruler for visualization

In [131]:
#Initialize NER ruler

ruler = nlp.add_pipe("entity_ruler", before = "ner")

In [132]:
#Add weakness patterns to NER ruler
for item in weakness_pattern:
  ruler.add_patterns([{"label": "WEAKNESS TERM", "pattern": item}])

#Add sarcopenia diagnosis to NER ruler
for i in sarcopenia_diagnosis:
  ruler.add_patterns([{"label": "SARCOPENIA", "pattern": i}])

#Add SARC-F to NER ruler
for i in sarcf_pattern:
  ruler.add_patterns([{"label": "SARC-F", "pattern": i}])


In [133]:
#Apply nlp pipeline to letter A
doc_A = nlp(letter_A)

/usr/local/lib/python3.7/dist-packages/scispacy/abbreviation.py:230: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


In [134]:
#Visualise weakness entities in Letter A

def get_entity_options():
  entities = ["WEAKNESS TERM", "SARCOPENIA", "SARC-F"]
  colors = {"WEAKNESS TERM": 'linear-gradient(90deg, #ffff66, #ff6600)', "SARCOPENIA": 'linear-gradient(90deg, #aa9cfc, #fc9ce7)',
            "SARC-F": 'linear-gradient(180deg, #66ffcc, #abf763)'}
  options = {"ents": entities, "colors": colors}
  return options
options = get_entity_options()

displacy.render(doc_A, style = 'ent', options=options, jupyter = True)

In [135]:
#Apply nlp pipeline to letter B
doc_B = nlp(letter_B)

/usr/local/lib/python3.7/dist-packages/scispacy/abbreviation.py:230: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


In [136]:
#Visualise weakness entities in Letter B

def get_entity_options():
  entities = ["WEAKNESS TERM", "SARCOPENIA", "SARC-F"]
  colors = {"WEAKNESS TERM": 'linear-gradient(90deg, #ffff66, #ff6600)', "SARCOPENIA": 'linear-gradient(90deg, #aa9cfc, #fc9ce7)',
            "SARC-F": 'linear-gradient(180deg, #66ffcc, #abf763)'}
  options = {"ents": entities, "colors": colors}
  return options
options = get_entity_options()

displacy.render(doc_B, style = 'ent', options=options, jupyter = True)

# Negation detection

In [137]:
#Define termset as clinical
ts = termset("en_clinical_sensitive")

#Add negex to nlp pipeline
nlp.add_pipe("negex", config={
    "ent_types":["SARCOPENIA","WEAKNESS TERM","SARC-F"],
    "neg_termset":ts.get_patterns()
})

In [138]:
#View termset patterns in use
print(ts.get_patterns())

{'pseudo_negations': ['not able to be', 'not certain if', 'not certain whether', 'not necessarily', 'without any further', 'without difficulty', 'without further', 'might not', 'not only', 'no increase', 'no significant change', 'no change', 'no definite change', 'not extend', 'not cause', 'gram negative', 'not rule out', 'not ruled out', 'not been ruled out', 'not drain', 'no suspicious change', 'no interval change', 'no significant interval change'], 'preceding_negations': ['rule the patient out', 'cannot', "don't", 'werent', "can't", 'without any reactions or signs of', "doesn't", 'if you get', 'evaluate for', 'teach the patient', 'rule patient out', 'history of', 'declined', 'never had', 'h/o', 'no signs of', 'not', 'symptoms atypical', 'arent', 'ruled her out', 'versus', 'not demonstrate', 'leads to', 'fails to reveal', 'supposed', 'patient was not', 'cant', 'no sign of', 'monitored for', 'educated the patient', 'never developed', 'ruled him out', 'dont', 'taught the patient', "wa

In [139]:
#No further does not seem to fit as a pseudo-negation, therefore I will remove it, and add to preceding negation instead
ts.remove_patterns({"pseudo_negations": ["no further"]})
ts.add_patterns({"preceding_negations": ["no further"]})

In [140]:
#Also add "blood pressure" as a preceding negation, sometimes a clinician may refer to a "fall" in blood pressure, this does not mean that a patient has had a fall.
#Also add "no"
ts.add_patterns({"preceding_negations": ["blood pressure", "BP", "no"]})

In [141]:
#Check that termset has been modified
print(ts.get_patterns())

{'pseudo_negations': ['not able to be', 'not certain if', 'not certain whether', 'not necessarily', 'without any further', 'without difficulty', 'without further', 'might not', 'not only', 'no increase', 'no significant change', 'no change', 'no definite change', 'not extend', 'not cause', 'gram negative', 'not rule out', 'not ruled out', 'not been ruled out', 'not drain', 'no suspicious change', 'no interval change', 'no significant interval change'], 'preceding_negations': ['rule the patient out', 'cannot', "don't", 'werent', "can't", 'without any reactions or signs of', "doesn't", 'if you get', 'evaluate for', 'teach the patient', 'rule patient out', 'history of', 'declined', 'never had', 'h/o', 'no signs of', 'not', 'symptoms atypical', 'arent', 'ruled her out', 'versus', 'not demonstrate', 'leads to', 'fails to reveal', 'supposed', 'patient was not', 'cant', 'no sign of', 'monitored for', 'educated the patient', 'never developed', 'ruled him out', 'dont', 'taught the patient', "wa

In [142]:
# View any negations in letter A, True indicates a negation
for e in doc_A.ents:
  print(e.text, e._.negex)

NE4 False
Poor mobility False
chronic pain False
falls False
Weight loss False
anaemia False
Low mood False
poor mobility False
Breathlessness False
Chronic back pain False
incontinence False
epileptic seizures False
Hypertension False
Atrial fibrillation False
Asthma 

Patent foramen ovale False
cerebellar stroke False
Atorvastatin False
Docusate False
fumarate False
Vitamin-D3 False
Furosemide False
Gaviscon False
inhaler False
Salbutamol False
Lansoprazole False
Losartan False
Paracetamol False
Phenytoin False
Tegretol False
Codeine False
Warfarin False
codeine False
poor mobility False
falls False
fallen False
lack of mobility False
housebound False
lacks strength False
falling False
pain False
fall False
vertigo False
pain False
pain False
appetite False
nausea or vomiting False
constipation False
toothache False
breathlessness False
cough False
phlegm False
rash False
rash False
crampy False
abdominal pain False
ideation False
jaundice False
anaemia False
cyanosis False
clubbing 

In [143]:
# View any negations in letter B, True indicates a negation
for e in doc_B.ents:
  print(e.text, e._.negex)

Newcastle NE2 5DH

 False
Falls False
gait and balance disorder False
Hyperthyroidism False
thyroxine False
Sarcopenia False
Orthostatic hypotension False
Hypertension False
Hypothyroidism False
Vitamin B12 False
fractured wrist

Visual impairment False
cataracts False
Alendronic acid False
Calcium False
vitamin-D 

Bendroflumethiazide False
Vitamin B12 False
Simvastatin False
Ramipril False
thyroxine False
falls False
falls False
fell False
lightheadedness False
vertigo False
palpitations False
chest pain False
syncopal False
fell False
appetite False
choking False
dysphagia False
nausea False
abdominal pain False
jaundice False
anaemia False
clubbing False
cyanosis False
lymphadenopathy False
goitre False
ankle oedema False
organomegaly False
bradykinesia False
tremor False
exophthalmos False
Sarc-F score of 7/10 False
falling False
174/64 False
fell False
falls False
sarcopenia False
falls False
syncope False
appetite False
weight loss False
hyperthyroid False
thyroxine False
sarcop

No negative entities in letters at present. There will need to ammend letters to include some negative entities for detection. 

In [144]:
#Create letter with some negative entities that have been added
letter_nA = '''Diagnoses: Poor mobility due to chronic pain, low confidence and previous falls. Possibly have sarcopenia.
The patient report no chronic pain,
Weight loss, anaemia and raised inflammatory markers of unknown aetiology 
Low mood secondary to poor mobility 
Breathlessness and elevated BNP awaiting echo 
Chronic back pain with degenerative changes on MRI 
Urinary frequency and incontinence 

The patient shows no difficulty walking.

Thank you for referring Mr Smith who attended for a face-to-face assessment at the Belsay Clinic accompanied by his niece today. He gives a history of poor mobility and falls; both these problems are longstanding and indeed he was assessed at the Belsay clinic by my colleague Dr Boyle back in 2019 and had a course of physiotherapy at the time. More recently he has lost confidence, his balance is worse and he has fallen more. He is clear that the lack of mobility is his greatest frustration; he is housebound unless he can be accompanied out of the house by his niece and even then he gets out only in a wheelchair. He thinks that a combination of things are stopping him being more mobile: he feels that he lacks strength, has a fear of falling and also has pain in the front and back of his legs. This is worse at night and keeps him awake at times. His last fall was two months ago. He notes feeling unsteady on standing but on close questioning this sensation did not appear to be consistent with vertigo. 
He also complains of pain across her shoulders starting in the right arm and going across the shoulders to the left arm. This has been present for about six months and is not related to exertion. It is no worse in the morning and he does not feel particularly stiff. He also describes stabbing pain over his right eye that then migrates over the top of his head. This does not seem to be related to stressful events. He has lost a considerable amount of weight - 8kg from March to August this year, and another five kg since as his weight in clinic today was 71.5 kg. His appetite is not as good as it usually is but he denies nausea or vomiting; he avoids constipation by taking laxatives. He does not complain of toothache, does not choke on food or drink and says that he eats reasonably well. He has seen a dietitian. He complains of occasional breathlessness and cough at night and brings up a small quantity of phlegm but this is little changed. He has not noticed any blood in his stools. He complains of a rash on his legs, more on the right than the left, and that her legs are often cool. He notes that this rash has been present for at least 10 years and I also note recent vascular duplex studies that suggest good arterial flow in the legs. He has urinary frequency and is often not aware of when he needs to go to the loo; he also complains of a few minutes of crampy lower abdominal pain after micturition. Perhaps unsurprisingly he is somewhat low in mood but still enjoys going out and seeing people. He denies suicidal ideation or early morning waking. He admits that he tends to live in the past more nowadays. He does not complain of any subjective memory problems. 
On examination, he was alert and engaged well with the consultation. There was no jaundice, anaemia, cyanosis, clubbing or lymphadenopathy. There was a confluent discolouration on both lower legs which was cool to the touch, not raised or tender and was not blanching. Similar changes were seen above the knees but looked more petechial in nature. Heart sounds were normal, JVP was not raised and the chest was clear. Abdominal examination revealed a 2cm liver edge but no ascites or masses. There was an old scar noted on his left upper arm where a basal cell carcinoma has been removed previously. He was mildly tender across the shoulders and in the thoracic paraspinal muscles; tenderness was not confined to the spine. He was able to raise her arms above her head. Neurological examination revealed normal tone, power and coordination. Cranial nerve examination was normal with no nystagmus. He was able to rise unaided from a chair quite quickly but was very reluctant to take a step forward. However he could walk a few paces with support from one person albeit unsteadily. There was no bradykinesia or tremor noted. 
His SarcScreen revealed a SARC-F score of 10/10, and Fried frailty score of 2/5 denoting prefrailty. He was unable to attempt the 3m walk but his maximum grip strength was 22 kg. GDS was 10/15 and MMSE was 28/30. An active stand showed a blood pressure of 141/85 with no significant drop although he was unsteady on standing. His 12 lead electrocardiogram showed atrial fibrillation at 80 per minute with nil else of note. 
There are clearly a complex set of interlocking problems here. He is rather stronger than he thinks he is on although his balance is poor, I think it is a lack of confidence rather than a lack of strength that is preventing him from mobilising more. He accepts this and is happy for us to refer to physiotherapy for strength and balance training which I think will help build his confidence. His mood is low but not sufficiently low to denote depression and I think if we can improve his mobility, his mood will improve. Some of his chronic back pain is undoubtedly due to the degenerative changes seen on MRI and I understand that surgery is not going to be an option for this. I would be reluctant to change her painkillers much at the moment though given that he is already on antiepileptic medications and so adding in other agents for chronic pain may not help much but might interact with these medications. 
We discussed the fine balance between benefits and side effects of all of these medications today. What would perhaps be helpful though is to reduce the codeine to 15 mg so that he can try and take this more regularly; he finds that the 30 mg dose makes him feel very unsteady on her feet. Having said all of this, it is clear that the blood tests suggest some issues that require further investigation. His breathlessness on exertion and raised BNP suggest that investigation for LV systolic dysfunction causing heart failure would be beneficial and I note that he has already had an echo requested. His weight loss is quite dramatic and this, together with the admittedly longstanding rash on his legs, anaemia, raised ESR and high platelet counts suggest that there is an inflammatory disorder present. Whether this is autoimmune or due to another aetiology is unclear. I have requested bloods today including repeat U&Es, liver function tests, calcium, myeloma screen, CRP and ESR, autoimmune screen, creatine kinase and iron studies. I have also requested an ultrasound of the abdomen to investigate the liver edge that I could feel; I note that a recent chest x-ray showed normal heart size and clear lung fields. Despite the possible finding of horizontal nystagmus by Community Nursing colleagues I could not find any evidence of this today and I don’t think we need to progress to brain scanning at present. 
Once we have got the blood and ultrasound results, I will phone Mr Smith again and discuss the best way forward. It may be that if we do not find another cause for his inflammatory condition, a trial of steroids might be warranted as some of the features including the shoulder discomfort could be consistent with polymyalgia rheumatica. I have explained this today but also explained that I think we need to investigate further before diving in with treatment in case this is not the diagnosis. "
'''

#apply nlp pipeline to letter_nA
doc_nA = nlp(letter_nA)

/usr/local/lib/python3.7/dist-packages/scispacy/abbreviation.py:230: UserWarning: [W036] The component 'matcher' does not have any patterns defined.
  global_matches = self.global_matcher(doc)


In [145]:
# Highlight any negative entities within text:

def add_neg_entities(doc):
  new_ents = []
  for ent in doc.ents:
    #only check for entity if negex is true
    if ent._.negex:
      ent.label_="NEGATION"

    new.ents.append(ent)

  doc.ents = new_ents
  return doc

  #apply above function to doc_nA
  doc_nA = add_neg_entities(doc_nA)

In [146]:
#Visualise negations in doc_nA

def get_entity_options_negations(): 
  entities = ["NEGATION"]
  colors = {"NEGATION":'linear-gradient(0deg, rgba(255,0,0,0), rgba(255,0,0,1))'}
  options = {"ents": entities, "colors": colors}
  return options
options = get_entity_options_negations()

displacy.render(doc_nA, style="ent", options = options, jupyter = True)